Kanishk Mair (kmair@andrew.cmu.edu)
Date: 2018-12-10 12:52:22.690955


- ASSIGNMENT: exam3-3
- POINTS: 3
- CATEGORY: exam-3
- RUBRIC: default
- RUBRIC_CATEGORIES: technical, presentation
- RUBRIC_WEIGHTS: 0.8, 0.2
- DUEDATE: 2018-12-10 23:59:59
- GRADER: John Kitchin


**This is an exam. You must be present in the exam room to get credit for this problem unless you have prior permission from the instructor. You may not talk during the exam except to ask an instructor a question. By turning this in, you agree that this work is your own, and you did not get unauthorized help to complete it or provide unauthorized help to anyone else. You may not modify your exam answer after the due time without permission.**

A common problem in solving nonlinear problems is *how to make the initial guess*?

Let's consider finding the solution to the following nonlinear equations:

$2 + x + y - x^2 + 8 x y + y^3 = 0$

$1 + 2x - 3y + x^2 + xy - y e^x = 0$

The strategy we work on here is to reformulate these equations with a new variable $\lambda$

$2 + x + y + \lambda(- x^2 + 8 x y + y^3) = 0$

$1 + 2x - 3y + \lambda(x^2 + xy - y e^x) = 0$




# Part 1 solve the linear problem



If $\lambda=1$ then we have the original nonlinear equations. If you set $\lambda=0$ though, you have a simple linear set of equations to solve. Find a solution to those equations for $\lambda=0$:



In [7]:
from scipy.optimize import fsolve
import numpy as np

def fn(X):
    x, y = X
    f1 =  2 + x + y
    f2 = 1 + 2*x - 3* y 
    return [f1, f2]

l = 0
ans = fsolve(fn, [0,1])
print(f'For λ = 0, the values of x = {ans[0]:1.1f} and y = {ans[1]:1.1f}.')

For λ = 0, the values of x = -1.4 and y = -0.6.


This solution represents the solution to the equations when $\lambda=0$. If we could derive a set of equations for $\frac{dx}{d\lambda}$ and $\frac{dy}{d\lambda}$, then we can treat this linear solution as an initial value, and integrate the ODEs from $\lambda=0$ to $\lambda=1$ to find the solution to the nonlinear equations. In what follows, we motivate how to derive those equations.




# Part 2 formulate a system of ODEs to solve the nonlinear problem



Next, we consider the equations as

$f(x, y) = 2 + x + y + \lambda(- x^2 + 8 x y + y^3) = 0$

$g(x, y) = 1 + 2x - 3y + \lambda(x^2 + xy - y e^x) = 0$

from calculus, you can show that:

$\frac{\partial f}{\partial x}\frac{\partial x}{\partial \lambda}+\frac{\partial f}{\partial y}\frac{\partial y}{\partial \lambda}=-\frac{\partial f}{\partial \lambda}$

$\frac{\partial g}{\partial x}\frac{\partial x}{\partial \lambda}+\frac{\partial g}{\partial y}\frac{\partial y}{\partial \lambda}=-\frac{\partial g}{\partial \lambda}$

You can rewrite this in a linear algebra form as:

\begin{equation}
\left[\begin{array}{cc}
\frac{\partial f}{\partial x} \frac{\partial f}{\partial y} \\
\frac{\partial g}{\partial x} \frac{\partial g}{\partial y}
\end{array}\right]
\left[\begin{array}{c}
\frac{\partial x}{\partial \lambda}\\
\frac{\partial y}{\partial \lambda}
\end{array}\right]
=
\left[\begin{array}{c}
-\frac{\partial f}{\partial \lambda}\\
-\frac{\partial g}{\partial \lambda}
\end{array}\right]
\end{equation}

The matrix on the left is the Jacobian of $F = [f(x,y), g(x, y)]$. This means you can solve for:

$$\left[\begin{array}{c}
\frac{\partial x}{\partial \lambda}\\
\frac{\partial y}{\partial \lambda}
\end{array}\right]
=
\mathbf{J}^{-1}
\left[\begin{array}{c}
-\frac{\partial f}{\partial \lambda}\\
-\frac{\partial g}{\partial \lambda}
\end{array}\right]$$

This last equation defines a set of differential equations that can be integrated from $\lambda=0$ where we know what (x, y) are, to $\lambda=1$ which leads to a solution to the original set of nonlinear equations!

Use the last equation to define a function for a system of ODEs, and then integrate the system of ODES from $\lambda=0$ to $\lambda=1$ to find the solution to the nonlinear set of equations. The solution is the value of $x, y$ at $\lambda=1$.



In [8]:
from autograd import jacobian, elementwise_grad
import autograd.numpy as np

def f(x,y,lam):
    return 2 + x + y + lam * (-x**2 + 8*x*y + y**3)

def g(x,y,lam):
    return 1 + 2 * x - 3* y + lam * (x**2 + x * y - y * np.exp(x))

dfdl = elementwise_grad(f, 2)
dgdl = elementwise_grad(g, 2)

dfdl(1.0,1.0,1.0)

8.0

In [11]:
def F(X):
    x, y = X
    f1 = 2 + x + y + 1 * (-x**2 + 8*x*y + y**3)
    f2 = 1 + 2 * x - 3* y + 1 * (x**2 + x * y - y * np.exp(x))
    return np.array([f1, f2])

Jf = jacobian(F)
lam = np.linspace(0,1)


x0 = -1.4
y0 = 0.6
Jinv = np.linalg.inv(Jf(np.array([x, y])))

dFl = np.array([-dfdl(x,y,lam), -dgdl(x,y,lam)])

dxy = Jinv @ dFl

dx= dxy[0]
dy = dxy[1]

dx
# from scipy.integrate import quad

# X = quad()

NameError: name 'x' is not defined

In [18]:
from scipy.integrate import quad

def F(X):
    x, y = X
    f1 = 2 + x + y + 1 * (-x**2 + 8*x*y + y**3)
    f2 = 1 + 2 * x - 3* y + 1 * (x**2 + x * y - y * np.exp(x))
    return np.array([f1, f2])

Jf = jacobian(F)


x0 = -1.4
y0 = 0.6

def integrand(P):
    x0, y0, lam0 = P
    dl = 0.01
    l1 = lam0
    xi = x0
    for i in range(100):
        
        l2 = l1 + dl
        Jinv = np.linalg.inv(Jf(np.array([x, y])))
        dFl = np.array([-dfdl(x,y,lam), -dgdl(x,y,lam)])

        dxy = Jinv @ dFl

        dx= dxy[0]
        dy = dxy[1]

        X = quad(dx, l1, l2)
        x = xi + X

lamb = np.linspace(0,1)

integrand(np.array([x0, y0, 0]))
# integrand(np.array([x, y]), lamb)

UnboundLocalError: local variable 'x' referenced before assignment


# Part 3 Verify the solution you found



Use a method of your choice to verify your solution from Part 2.



In [14]:
def f(X):
    x, y = X
    f1 =  2 + x + y - x**2 + 8*x*y + y**3
    f2 = 1 + 2*x - 3* y + x**2 + x * y - y * np.exp(x)
    return [f1, f2]

l = 0
ans = fsolve(f, [0,1])
print(f'The ans by fsolve is: x = {ans[0]:1.3f} and y = {ans[1]:1.3f}.')


The ans by fsolve is: x = -1.000 and y = -0.000.
